### Node

In [1]:
import math
class Node():
    def __init__(self, data):
        self.label = data[0]
        self.parent=data[1] #maintains the label of parent node
        self.parentNode=None #maintains the actual parent node
        self.children=[]
        self.posTrue=0 #posterior probability 
        self.posFalse=0
        self.updated=False #to maintain whether the node was visited
        self.evidence=False #for evidence
        self.pTrue=float(data[2]) #prior probability
        self.margTrue=float(data[2]) #marginal probability
        if self.parent=='null':
            self.pFalse=self.margFalse=1-self.pTrue

        else:
            self.pFalse=float(data[3])
            self.margFalse=float(data[3])
    
    def getLabel(self):
        return self.label

    def getParent(self):
        return self.parent
    
    def addChild(self,data):
        self.children.append(data)
            
    def calculateMarginal(self):
        if self.parent!='null':
            self.margTrue=round(self.pTrue*(self.parentNode.margTrue)+self.pFalse*(self.parentNode.margFalse),2)
            self.margFalse=round(1-self.margTrue,2)
        return (self.label,self.margTrue,self.margFalse)
    
   #calculates posterior of children
    def calculatePosterior(self):
        for i in self.children:
            if i.evidence!=True: 
                if i.updated!=True:
                    i.posTrue = self.posFalse * i.pFalse + self.posTrue * i.pTrue 
                    i.posFalse = 1 - i.posTrue
                    if (len(i.children) != 0):
                        i.calculatePosterior()
        return 

    def getParentNode(self):
        return self.parentNode
    
    def setParentNode(self,node):
        self.parentNode=node
    #finds posterior of children
    def findPosterior(self):
        lst = lst1=[]
        for i in self.children:
            if (len(i.children)!=0):
                lst1 = i.findPosterior()
            lst.append((i.label, round(i.posTrue,3),round(1-i.posTrue,3)))
            lst.extend(lst1) 
            i.updated=False
        cleanlist = []
        [cleanlist.append(x) for x in lst if x not in cleanlist]
        return cleanlist


### Network

In [2]:
class Network():
    def __init__(self,file):
            lst=[]
            self.nodes=[]
            self.root=None
            self.totalNodes=0
            
            #reading file
            file=open(file,'r')
            for line in file:
                data=line.split(',')
                data=[i.rstrip() for i in data]
                lst.append(data)
            lst.pop(0)
            totalNodes=len(lst)
            file.close()
            for i in lst:
                if i[1]=='null':
                    self.root=Node(i)
                    self.nodes.append(self.root)
                else:
                    self.nodes.append(Node(i))
                    
            #to add children nodes and parent node
            for i in self.nodes:
                if i.getParent()!='null':
                    for j in range(len(self.nodes)):
                        if self.nodes[j].getLabel()==i.getParent():
                            self.nodes[j].addChild(i)
                            i.setParentNode(self.nodes[j])
                            
    def getNodeList(self):
        return self.nodes


### Part (a)

In [3]:
#part a
net=Network("Data.txt")
lst=net.getNodeList()
for i in lst:
    print(i.getLabel(),end='')



ABCD

### Part (b)

In [4]:
#part b
def printMarginal(net):
    lst=net.getNodeList()
    marginal=[]
    for i in lst:
        s=i.calculateMarginal()
        string='Node '+s[0]+': '+'True '+str(s[1])+' False '+str(s[2])
        marginal.append(string)
    print (marginal)
printMarginal(net)


['Node A: True 0.6 False 0.4', 'Node B: True 0.44 False 0.56', 'Node C: True 0.52 False 0.48', 'Node D: True 0.61 False 0.39']


### Part (c)

In [5]:
#part c

def makeInference(net,label):
    lst=net.getNodeList()
    temp=None
    for i in lst:
        if i.getLabel()==label:
            temp=i
    temp.evidence=True
    temp.posFalse=0
    temp.posTrue=1
    temp.calculatePosterior()
    if(temp.parent!="null"):
        traverse(net,temp)
    lst=net.root.findPosterior()
    lst.append((net.root.label,round(net.root.posTrue,3),round(1-net.root.posTrue,3)))
    net.root.updated=False
    return sorted(lst)

#traverse function to traverse the parent nodes and the remaining child nodes of the parent
def traverse(net,node):
    if node.parent!="null":
        temp=node
        curr=None
        
        #parent traversal-> to update prosterior probability
        parent=node.parentNode
        if parent.evidence!=True and parent.updated!=True:
            parent.posTrue=((temp.pTrue*temp.posTrue*parent.margTrue)/temp.margTrue)+(
                (1-temp.pTrue)*(temp.posFalse*parent.margTrue)/temp.margFalse)
            parent.posFalse=1-parent.posTrue
            parent.updated=True
            lst=net.getNodeList()
            for i in lst:
                if i.getLabel()==parent.getLabel():
                    curr=i
            traverse(net,curr)
            
            #children traversal-> to update prosterior probability
            for i in parent.children:
                if i.evidence!=True and i.updated!=True:
                    i.posTrue=i.pTrue*parent.posTrue+i.pFalse * parent.posFalse
                    i.posFalse=1-i.posTrue
                    i.calculatePosterior()
                    i.updated=True
                    traverse(net,i)
        return
infer=makeInference(net,"D")
for i in infer:
    print('Node '+i[0]+':'+' posterior true->'+str(i[1])+' posterior false->'+str(i[2]))

Node A: posterior true->0.766 posterior false->0.234
Node B: posterior true->0.507 posterior false->0.493
Node C: posterior true->0.767 posterior false->0.233
Node D: posterior true->1 posterior false->0


In [6]:
# part d
print(makeInference(net,"A"))



[('A', 1, 0), ('B', 0.6, 0.4), ('C', 0.8, 0.2), ('D', 1, 0)]
